In [6]:
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rand

In [2]:
hypercube = stats.qmc.LatinHypercube(2)

In [3]:
print(hypercube.random(100))

[[0.55482618 0.08429964]
 [0.61736808 0.47852338]
 [0.16954878 0.05082605]
 [0.35127654 0.9900567 ]
 [0.77869064 0.59486658]
 [0.90083466 0.07759966]
 [0.39222227 0.00972354]
 [0.43186561 0.03356768]
 [0.05630171 0.66581769]
 [0.70572845 0.18865713]
 [0.9932031  0.82554352]
 [0.26866927 0.41832641]
 [0.89774519 0.31242876]
 [0.98099627 0.69122674]
 [0.40300055 0.33117127]
 [0.57228067 0.11959954]
 [0.36148608 0.40606316]
 [0.33610897 0.57711209]
 [0.24149837 0.96945134]
 [0.21921952 0.98784553]
 [0.18671634 0.58439541]
 [0.95109164 0.92501279]
 [0.63588012 0.88715125]
 [0.83885006 0.39994185]
 [0.08670037 0.27687664]
 [0.07045101 0.64706495]
 [0.46273875 0.29302184]
 [0.37886703 0.5073328 ]
 [0.50422516 0.24831508]
 [0.79993487 0.22266781]
 [0.560277   0.19835631]
 [0.88138486 0.35308483]
 [0.09764959 0.12579262]
 [0.04294221 0.8647025 ]
 [0.4733783  0.44624246]
 [0.27495227 0.91738219]
 [0.13349941 0.76490524]
 [0.02603096 0.53200885]
 [0.11445726 0.78105502]
 [0.42009535 0.81242596]


In [ ]:
def check_converge(z_0, treshold=2, max_iterations=25):
    """
    Checks if the complex number z_0 converges within the max_iterations.
    """

    z = z_0
    counter = 0
    while True:
        z = z ** 2 + z_0
        counter += 1

        if abs(z) > 2:
            return False
        if counter > max_iterations:
            return True

def determine_area(i = 25, s=10000, sample_method="pure"):
    """
    Determine the area of the mandelbrot set.

    Parameters:
    i: int, Max number of iterations
    s: int, The number of sample points.
    sample_method: string, options are:
        'pure': pure random sampling,
        'hypercube': hypercube sampling,
        'orthogonal': orthogonal sampling
    """

    xmin = -2
    xmax = 1
    ymin = -2
    ymax = 2
    bounding_box_area = (xmax - xmin) * (ymax - ymin)

    if sample_method == "pure":
        Z = generate_sample_points_pure_random(s, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
    elif sample_method == "hypercube":
        Z = generate_sample_points_hypercube(s, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)
    elif sample_method == "orthogonal":
        Z = generate_sample_points_orthogonal(s, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)

    Z0 = Z

    N = np.zeros(s)

    for _ in range(i):
        Z = np.where(abs(Z) < 2, Z ** 2 + Z0, 100)

    N[Z != 100] = 1

    n_good_points = sum(N)
    
    return bounding_box_area * (n_good_points / s)

def plot_i_s_contour(i_list, s_list, log=False):
    """
    Plots the difference between the expected and calculated area for different values for i and s. 
    i is the maximum number of iterations and s is the number of samples. 
    """

    total_evaluations = len(i_list) * len(s_list)
    n_evaluations = 0

    expected_area = 1.506484

    data = []
    for i in i_list:

        results = []
        for s in s_list:
            results.append(determine_area(i=i, s=s) - expected_area)
            n_evaluations += 1

            if log and n_evaluations % 10 == 0:
                print(f"Progress: {n_evaluations/total_evaluations * 100:0f}%", end="\r")
        
        data.append(results)

    contour = plt.contourf(s_list, i_list, data, 10, cmap="PuBuGn_r")
    
    colorbar = plt.colorbar(contour)
    colorbar.set_label("Difference between expected area")

    plt.xlabel("Samples")
    plt.ylabel("Max iterations")

    plt.xscale("log")
    plt.yscale("log")
    plt.show()

def plot_i_s_variance(i_list, s_list, nruns=10, log=False):
    """
    Plots the variance of the difference between the expected and calculated area for different values for i and s. 
    i is the maximum number of iterations and s is the number of samples. 
    """
    
    total_evaluations = len(i_list) * len(s_list) * nruns
    n_evaluations = 0

    expected_area = 1.506484

    data = []
    for i in i_list:

        results = []
        for s in s_list:

            run_results = []
            for run in range(nruns):
                run_results.append(determine_area(i=i, s=s))
                n_evaluations += 1

                if log and n_evaluations % 10 == 0:
                    print(f"Progress: {n_evaluations/total_evaluations * 100:1f}%", end="\r")

            results.append(np.var(run_results))
        
        data.append(results)

    contour = plt.contourf(s_list, i_list, data, 10, cmap="PuBuGn_r")
    
    colorbar = plt.colorbar(contour)
    colorbar.set_label("Variance in area")

    plt.xlabel("Samples")
    plt.ylabel("Max iterations")

    plt.xscale("log")
    plt.yscale("log")
    plt.show()

def generate_sample_points_pure_random(s=10000, xmin=-2, xmax=1, ymin=-2, ymax=2):
    
    X = rand.rand(s) * (xmax - xmin) + xmin
    Y = rand.rand(s) * (ymax - ymin) + ymin

    return X + Y * 1j

def generate_sample_points_hypercube(s=10000, xmin=-2, xmax=1, ymin=-2, ymax=2):
    sampler = stats.qmc.LatinHypercube(2)
    points = sampler.random(s)

    l_bounds = [xmin, ymin]
    u_bounds = [xmax, ymax]

    points = stats.qmc.scale(points, l_bounds, u_bounds)

    return points[:,0] + points[:,1] * 1j

def generate_sample_points_orthogonal(s=10000, xmin=-2, xmax=1, ymin=-2, ymax=2):
    # TODO

    pass

In [28]:
pure = determine_area(i = 2500, s=10000000, sample_method="pure")


In [29]:
hyper = determine_area(i = 2500, s=10000000, sample_method="hypercube")

In [30]:
print(pure, hyper)

1.5068291999999999 1.5098016


In [5]:
borders = -0.743643135 + 0.121825963j, 0.000014628
